# NFL Arrests

* Arrest average per team. - Issac
* Time start vs. number of arrests. - Issac
* Week number vs. number of arrests - Mike
* Home team vs away team - Eric
* division game vs non division game - Eric

####  Little more challenging
* Team made playoffs vs Teams that missed playoffs - Need external dataset.
* Team average vs number of arrests - Mike
* Avg win/loss by timeframe - Issac

In [3]:
import numpy as np
import pandas as pd

In [4]:
nfl_arrests = pd.read_csv('ArrestSet.csv')

FileNotFoundError: File b'ArrestSet.csv' does not exist

In [5]:
nfl_arrests

NameError: name 'nfl_arrests' is not defined

In [5]:
nfl_arrests.groupby('season').mean()

,week_num,home_score,away_score,arrests
season,,,,
2011,9.095477,23.668342,20.251256,6.214660
2012,9.140000,23.420000,21.370000,7.020833
2013,9.179612,24.859223,21.509709,6.500000
2014,9.039216,24.240196,20.906863,6.750000
2015,9.126904,23.441624,21.532995,6.338624
